In [1]:
import numpy as np
import pandas as pd
from random import randrange
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from ograyspy_class import Ograyspy, select_spectrum_from_folder_list
from spec_class import Spec

In [2]:

# 2022-out-7: Excelente espectro para testes, tenho usado ultimamente:
# a_pattern = 'Si/SI2018/SI11318.Chn'
# 2022-nov-16: outros espectros:
# a_pattern = "Filtros/2022/Cci/CCI1603-I.Chn"
# a_pattern = "Filtros/2022/Cci/CCI2302-I.Chn"

# a_pattern = "Eso_non_existe.Chn"

# 2022-Dez-21 Buscando outro espectro
# a_pattern = 'CCI1603-I'
# 2022-Dez-28 Este é um espectro mais recente:
# a_pattern = 'SI09722.Chn'
# 2022-Dez-28 Este é um espectro mais recente que posso comparar com saída do Genie:
# a_pattern = 'SI06322.Chn'

In [3]:
spectra_list_df = pd.read_pickle('WindowsI56611.pkl')

In [4]:
spectra_list_df

,info_plat,info_mach,info_syst,info_node,home_path,spectra_path,n_files,a_spec_ind,a_spec_name,reduced_f_name,files_list,reduced_names_files_list,spectra_pattern_names,pkl_folder_files
0,Windows-10-10.0.19045-SP0,AMD64,Windows,I56611,C:\Users\mmaduar,C:\Users\mmaduar\Nextcloud\Genie_Transfer\Si\S...,49,0,,,[C:\Users\mmaduar\Nextcloud\Genie_Transfer\Si\...,"[BGTESTE.Chn, FRASCO ORIGINAL SI084_22.Chn, SI...","[**/*.[Cc][Hh][Nn], **/*.[Ii][Ee][Cc]]",WindowsI56611.pkl


In [5]:
reduc_nms = spectra_list_df.reduced_names_files_list[0]
fil_lst = spectra_list_df.files_list[0]
spc_pth = spectra_list_df.spectra_path[0]

In [6]:
reduc_nms

['BGTESTE.Chn',
 'FRASCO ORIGINAL SI084_22.Chn',
 'SI00822.Chn',
 'SI00922.Chn',
 'SI01622.Chn',
 'SI01722.Chn',
 'SI01822.Chn',
 'SI01922.Chn',
 'SI02022.Chn',
 'SI02522.Chn',
 'SI02622.Chn',
 'SI02722.Chn',
 'SI02922.Chn',
 'SI03022.Chn',
 'SI03122.Chn',
 'SI03222.Chn',
 'SI03322.Chn',
 'SI03422.Chn',
 'SI03522.Chn',
 'SI03722.Chn',
 'SI04022 (cont em 31-05-22).Chn',
 'SI04422.Chn',
 'SI04722.Chn',
 'SI05122.Chn',
 'SI05322.Chn',
 'SI05522.Chn',
 'SI05622.Chn',
 'SI05822.Chn',
 'SI05922.Chn',
 'SI06122.Chn',
 'SI06322.Chn',
 'SI06522.Chn',
 'SI06722.Chn',
 'SI06922.Chn',
 'SI07222.Chn',
 'SI07322.Chn',
 'SI07522.Chn',
 'SI07722.Chn',
 'SI07922.Chn',
 'SI08222.Chn',
 'SI08422.Chn',
 'SI08522.Chn',
 'SI08622.Chn',
 'SI08722.Chn',
 'SI09122.Chn',
 'SI09222.Chn',
 'SI09322.Chn',
 'SI09422.Chn',
 'SI09722.Chn']

In [7]:
# 2022-Dez-28 Este é um espectro mais recente que posso comparar com saída do Genie:
a_pattern = 'SI06322.Chn'

In [8]:
f_name, reduced_f_name = select_spectrum_from_folder_list(reduc_nms, fil_lst, spc_pth, a_pattern)
f_name, reduced_f_name

Existing:
Achou! indice=30, nomearq = SI06322.Chn
name:  C:\Users\mmaduar\Nextcloud\Genie_Transfer\Si\SI2022\SI06322.Chn
Final choices:
spectra_path: C:\Users\mmaduar\Nextcloud\Genie_Transfer\Si\SI2022
a_spec_name: C:\Users\mmaduar\Nextcloud\Genie_Transfer\Si\SI2022\SI06322.Chn
reduced_f_name: SI06322.Chn


(WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer/Si/SI2022/SI06322.Chn'),
 'SI06322.Chn')

In [9]:
a_spec = Spec(f_name, reduced_f_name)
vars(a_spec)

{'net_spec_ser_an': None,
 'final_composed_baseline': None,
 'f_name': WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer/Si/SI2022/SI06322.Chn'),
 'reduced_f_name': 'SI06322.Chn',
 'sufx': '.chn',
 'spec_io': None,
 'lv_time': 50000.0,
 'rl_time': 50028.58,
 'source_datetime': None,
 'pkl_file': WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer/Si/SI2022/SI06322.pkl'),
 'origin_spec_ser_an': <generic_series_analysis_class.GenericSeriesAnalysis at 0x249244ffc70>,
 'start_datetime': datetime.datetime(2022, 8, 4, 19, 42, 23),
 'det_descr': 'MAQUINA_01 MCB 1 Input 1',
 'sam_descr': 'SI-063/22 - Agua da Piscina do Reator IEA-R1 (GMX)',
 'channel_energy_calib': <genericcalib_class.ChannelEnergyCalib at 0x249244ffa30>}

In [10]:
orig_ser = a_spec.origin_spec_ser_an
vars(orig_ser)

{'mix_regions': None,
 'spl_baseline': None,
 'eval_baseline': None,
 'y_s': array([0, 0, 0, ..., 3, 1, 2]),
 'nzero': array([False, False, False, ...,  True,  True,  True]),
 'n_ch': 4096,
 'final_baseline': None,
 'x_s': array([0.000e+00, 1.000e+00, 2.000e+00, ..., 4.093e+03, 4.094e+03,
        4.095e+03]),
 'chans_nzero': array([  26.,   27.,   28., ..., 4093., 4094., 4095.]),
 'counts_nzero': array([ 11, 221, 380, ...,   3,   1,   2]),
 'unc_y': array([ 3.31662479, 14.86606875, 19.49358869, ...,  1.73205081,
         1.        ,  1.41421356]),
 'given_variance': array([0, 0, 0, ..., 3, 1, 2]),
 'y_smoothed': array([0, 0, 0, ..., 3, 1, 2]),
 'fft_s': None,
 'is_reg': array([False, False, False, ..., False, False, False]),
 'xs_all_mplets': [],
 'ys_all_mplets': [],
 'ys_all_steps': [],
 'chans_in_multiplets_list': [],
 'calculated_step_counts': [],
 'xs_bl_in_reg': array([], dtype=float64),
 'ys_bl_in_reg': array([], dtype=float64),
 'xs_bl_out_reg': array([], dtype=float64),
 'ys_b

In [11]:
a_spec.total_analysis(gener_dataframe=True)
# A opção acima, gener_dataframe=True,  grava o pkl_file do espectro
vars(a_spec)

Starting Spec.total_analysis...
Finish Spec.total_analysis!


{'net_spec_ser_an': <generic_series_analysis_class.GenericSeriesAnalysis at 0x24935a81d90>,
 'final_composed_baseline': <generic_series_analysis_class.GenericSeriesAnalysis at 0x2492ef21eb0>,
 'f_name': WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer/Si/SI2022/SI06322.Chn'),
 'reduced_f_name': 'SI06322.Chn',
 'sufx': '.chn',
 'spec_io': None,
 'lv_time': 50000.0,
 'rl_time': 50028.58,
 'source_datetime': None,
 'pkl_file': WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer/Si/SI2022/SI06322.pkl'),
 'origin_spec_ser_an': <generic_series_analysis_class.GenericSeriesAnalysis at 0x249244ffc70>,
 'start_datetime': datetime.datetime(2022, 8, 4, 19, 42, 23),
 'det_descr': 'MAQUINA_01 MCB 1 Input 1',
 'sam_descr': 'SI-063/22 - Agua da Piscina do Reator IEA-R1 (GMX)',
 'channel_energy_calib': <genericcalib_class.ChannelEnergyCalib at 0x249244ffa30>}

In [12]:
vars(orig_ser)

{'mix_regions': array([[  24,   57],
        [ 114,  180],
        [ 236,  267],
        [ 278,  285],
        [ 328,  341],
        [ 352,  363],
        [ 375,  386],
        [ 393,  400],
        [ 443,  450],
        [ 458,  465],
        [ 477,  500],
        [ 524,  535],
        [ 537,  544],
        [ 552,  567],
        [ 576,  589],
        [ 596,  607],
        [ 612,  636],
        [ 669,  674],
        [ 678,  685],
        [ 711,  720],
        [ 721,  745],
        [ 759,  768],
        [ 800,  807],
        [ 812,  827],
        [ 834,  851],
        [ 926,  951],
        [ 954,  984],
        [ 989, 1018],
        [1047, 1054],
        [1076, 1089],
        [1102, 1138],
        [1155, 1177],
        [1182, 1202],
        [1213, 1232],
        [1249, 1266],
        [1275, 1294],
        [1298, 1317],
        [1326, 1350],
        [1366, 1379],
        [1406, 1425],
        [1447, 1462],
        [1491, 1504],
        [1534, 1547],
        [1580, 1598],
        [1637, 16

In [13]:
net_ser = a_spec.net_spec_ser_an
vars(net_ser)

{'mix_regions': array([[  25,   57],
        [ 122,  134],
        [ 139,  150],
        [ 231,  240],
        [ 393,  400],
        [ 479,  498],
        [ 525,  534],
        [ 577,  588],
        [ 712,  719],
        [ 834,  851],
        [ 955,  966],
        [ 998, 1009],
        [1076, 1089],
        [1108, 1123],
        [1125, 1136],
        [1156, 1169],
        [1193, 1200],
        [1250, 1265],
        [1326, 1341],
        [1367, 1378],
        [1448, 1461],
        [1492, 1503],
        [1534, 1547],
        [1580, 1598],
        [1827, 1848],
        [1919, 1934],
        [2027, 2040],
        [2180, 2195],
        [2264, 2281],
        [2390, 2407],
        [2465, 2476],
        [2606, 2623],
        [2832, 2843],
        [2888, 2903],
        [3025, 3036],
        [3438, 3463],
        [3607, 3624],
        [4001, 4028]], dtype=int64),
 'spl_baseline': None,
 'eval_baseline': None,
 'y_s': array([0.        , 0.        , 0.        , ..., 2.0245828 , 0.        ,
       

In [14]:
# Só para verificar se o pkl_file pode ser recuperado:
read_analysis_spec = pd.read_pickle(a_spec.pkl_file)

In [15]:
read_orig = read_analysis_spec['origin_spec_ser_an'][0]
vars(read_orig)

{'mix_regions': array([[  24,   57],
        [ 114,  180],
        [ 236,  267],
        [ 278,  285],
        [ 328,  341],
        [ 352,  363],
        [ 375,  386],
        [ 393,  400],
        [ 443,  450],
        [ 458,  465],
        [ 477,  500],
        [ 524,  535],
        [ 537,  544],
        [ 552,  567],
        [ 576,  589],
        [ 596,  607],
        [ 612,  636],
        [ 669,  674],
        [ 678,  685],
        [ 711,  720],
        [ 721,  745],
        [ 759,  768],
        [ 800,  807],
        [ 812,  827],
        [ 834,  851],
        [ 926,  951],
        [ 954,  984],
        [ 989, 1018],
        [1047, 1054],
        [1076, 1089],
        [1102, 1138],
        [1155, 1177],
        [1182, 1202],
        [1213, 1232],
        [1249, 1266],
        [1275, 1294],
        [1298, 1317],
        [1326, 1350],
        [1366, 1379],
        [1406, 1425],
        [1447, 1462],
        [1491, 1504],
        [1534, 1547],
        [1580, 1598],
        [1637, 16

In [16]:
read_net = read_analysis_spec['net_spec_ser_an'][0]
vars(read_net)

{'mix_regions': array([[  25,   57],
        [ 122,  134],
        [ 139,  150],
        [ 231,  240],
        [ 393,  400],
        [ 479,  498],
        [ 525,  534],
        [ 577,  588],
        [ 712,  719],
        [ 834,  851],
        [ 955,  966],
        [ 998, 1009],
        [1076, 1089],
        [1108, 1123],
        [1125, 1136],
        [1156, 1169],
        [1193, 1200],
        [1250, 1265],
        [1326, 1341],
        [1367, 1378],
        [1448, 1461],
        [1492, 1503],
        [1534, 1547],
        [1580, 1598],
        [1827, 1848],
        [1919, 1934],
        [2027, 2040],
        [2180, 2195],
        [2264, 2281],
        [2390, 2407],
        [2465, 2476],
        [2606, 2623],
        [2832, 2843],
        [2888, 2903],
        [3025, 3036],
        [3438, 3463],
        [3607, 3624],
        [4001, 4028]], dtype=int64),
 'spl_baseline': None,
 'eval_baseline': None,
 'y_s': array([0.        , 0.        , 0.        , ..., 2.0245828 , 0.        ,
       

In [17]:
# 2023-May-26
# Ok! Espectro gravado, depois lido, então vamos prosseguir
# com a identificação dos nuclídeos.
# Agora, posso ler e analisar um espectro, gravá-lo e depois, em outro momento,
# ler o pkl com a análise.

In [18]:
x_nz = a_spec.origin_spec_ser_an.chans_nzero
y_nz = a_spec.origin_spec_ser_an.counts_nzero

In [20]:
chans = net_ser.x_s
ys_net_counts = net_ser.y_s
peaks_net = net_ser.pk_parms.peaks
peaks_1 = orig_ser.pk_parms.peaks
counts = orig_ser.y_s
chans_nzero = orig_ser.chans_nzero
counts_nzero = orig_ser.counts_nzero
unc_y = orig_ser.unc_y
eval_y = orig_ser.y_smoothed
eval_bl = orig_ser.eval_baseline
fin_bl = orig_ser.final_baseline
inis = orig_ser.pk_parms.propts['left_bases']
fins = orig_ser.pk_parms.propts['right_bases']

In [30]:
# 2023-May-24
# A seguir, um gráfico resumido e com escala em cps, para o utente.

In [31]:
counts_nzero_cps = counts_nzero / a_spec.lv_time
unc_y_cps = unc_y / a_spec.lv_time
fin_bl_cps = fin_bl / a_spec.lv_time
counts_cps = counts / a_spec.lv_time

In [35]:
# prosseguir a partir daqui:

In [36]:
# 2023-May-24
# Implementing:
#  - nuclide labels
# https://plotly.com/python/text-and-annotations/
#  - and energy search in pandas
# https://stackoverflow.com/questions/63529555/get-list-of-occurrences-using-pandas

In [18]:
get_energy = a_spec.channel_energy_calib.get_energy

In [19]:
get_energy(1000)

609.0498553514481

In [20]:
# the service URL
livechart = "https://nds.iaea.org/relnsd/v0/data?"

In [21]:
import urllib.request

def lc_read_csv(url):
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    return pd.read_csv(urllib.request.urlopen(req))

In [26]:
# 2023-May-26
# Load data into a dataframe, related to nuclides in a custom list
nucl_iear1_list = ['24na','51cr','54mn','58co','60co','65zn','103ru','110ag','131i','132te',
                   '137cs','140ba','140la','187w','239np',
                   '40k','208Tl','212Pb','212Bi','214Pb','214Bi','228Ac']
frames = []
for nucl in nucl_iear1_list:
    df = lc_read_csv(livechart + "fields=decay_rads&nuclides=" + nucl + "&rad_types=g")
    df = df[pd.to_numeric(df['intensity'],errors='coerce').notna()]
    df.intensity = df['intensity'].astype(float)
    frames.append(df)
df_all = pd.concat(frames)

In [27]:
df_all

,energy,unc_en,intensity,unc_i,start_level_energy,end_level_energy,multipolarity,mixing_ratio,unc_mr,conversion_coeff,...,decay_%,unc_d,q,unc_q,d_z,d_n,d_symbol,ensdf_publication_cut-off,ensdf_authors,Extraction_date
0,472.2023,0.0014,99.90000,0.001,472.2071,0,[M3],,NaN,4.69E-4,...,99.95,,,NaN,11,13,Na,31-Mar-2022,M. Shamsuzzoha Basunia and Anagha Chakraborty,2023-05-26
1,996.8300,0.1,0.00210,0.0002,5235.21,4238.38,,,NaN,,...,100.00,,5515.677,21.0,12,12,Mg,31-Mar-2022,M. Shamsuzzoha Basunia and Anagha Chakraborty,2023-05-26
2,1368.6250,0.005,99.99400,0.002,1368.667,0,E2,,NaN,1.3E-5,...,100.00,,5515.677,21.0,12,12,Mg,31-Mar-2022,M. Shamsuzzoha Basunia and Anagha Chakraborty,2023-05-26
3,2754.0080,0.011,99.86700,0.01,4122.844,1368.667,E2,,NaN,,...,100.00,,5515.677,21.0,12,12,Mg,31-Mar-2022,M. Shamsuzzoha Basunia and Anagha Chakraborty,2023-05-26
4,2871.0000,1,0.00025,0.00004,4238.38,1368.667,M1+E2,-23,9,,...,100.00,,5515.677,21.0,12,12,Mg,31-Mar-2022,M. Shamsuzzoha Basunia and Anagha Chakraborty,2023-05-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,89.9540,,2.01000,0.19,,,,,,,...,100.00,,2123.8,26.0,90,138,Th,31-Dec-2012,KHALIFEH ABUSALEEM,2023-05-26
268,93.3470,,3.20000,0.3,,,,,,,...,100.00,,2123.8,26.0,90,138,Th,31-Dec-2012,KHALIFEH ABUSALEEM,2023-05-26
269,105.5660,,1.16000,0.11,,,,,,,...,100.00,,2123.8,26.0,90,138,Th,31-Dec-2012,KHALIFEH ABUSALEEM,2023-05-26
270,106.8940,,1.54000,0.15,,,,,,,...,100.00,,2123.8,26.0,90,138,Th,31-Dec-2012,KHALIFEH ABUSALEEM,2023-05-26


In [29]:
# counts_nzero_cps = counts_nzero / a_spec.lv_time
# unc_y_cps = unc_y / a_spec.lv_time
# fin_bl_cps = fin_bl / a_spec.lv_time
# counts_cps = counts / a_spec.lv_time
# en_kev_nzero = get_energy(chans_nzero)
# en_kev = get_energy(chans)
peaks_net_kev = get_energy(peaks_net)

NameError: name 'peaks_net' is not defined

In [ ]:
# https://stackoverflow.com/questions/63529555

In [ ]:
animals = ['cat', 'dog', 'hamster', 'dolphin']

In [63]:
df_animals_sets = pd.DataFrame(data={
    'id': [1,2,3,4,5],
    'animals': ['dog,cat','dog','cat,dolphin','cat,dog','hamster,dolphin']
})
df_animals_sets

,id,animals
0,1,"dog,cat"
1,2,dog
2,3,"cat,dolphin"
3,4,"cat,dog"
4,5,"hamster,dolphin"


In [65]:
df = df_animals_sets
df_final = (df.astype(str).assign(animals=df.animals.str.split(','))
                          .explode('animals').groupby('animals').id.agg(','.join)
                          .reset_index())
df_final

,animals,id
0,cat,"1,3,4"
1,dog,"1,2,4"
2,dolphin,"3,5"
3,hamster,5


In [66]:
peaks_net_kev

array([  16.95175254,   23.67319643,   74.3895458 ,   76.22266686,
         85.99931252,  141.60398471,  239.98148167,  296.19719422,
        321.24984872,  353.63498747,  434.90335453,  512.50547945,
        584.6082412 ,  610.88297641,  659.15516436,  679.31949604,
        688.48510134,  708.03839266,  728.81376469,  766.08722627,
        812.52629316,  836.35686696,  886.46217597,  912.73691118,
        939.01164639,  966.50846231,  970.78574479, 1117.43542969,
       1122.32375252, 1174.87322295, 1240.2545408 , 1334.35475528,
       1386.29318535, 1463.28426993, 1507.2791754 , 1595.26898634,
       1731.53098524, 1766.97132576, 1849.46177351, 2106.0987221 ,
       2206.92038047, 2450.72548163])